# Clase 34

Para una mejor visualización entrar al siguiente [link](https://nbviewer.jupyter.org/github/racsosabe/Miscelanea/blob/master/UPC/Clase%2034%20-%20Strings%20II.ipynb)

# Requisitos Previos

* Matemática Básica
* Matemática Discreta

# Trie

El *retrieval tree* o **Trie** es una estructura de datos que permite almacenar strings y trabajar con sus prefijos, de manera que se plantea un árbol en el que cada nodo representará un prefijo de alguna de las cadenas y cada arista es una transición etiquetada con la letra que corresponde.

Definamos un poco el árbol que se termina formando:

1. El conjunto de nodos $V$ será tal que existirá un nodo por cada prefijo de alguna de las cadenas (sin repeticiones) y la raiz del árbol representará a la cadena vacía.

2. El conjunto de aristas está definido de manera que cada una tendrá como etiqueta una letra $c$ y la arista $(u, v)$ con etiqueta $c$ existe si $prefix(u) + c = prefix(v)$ donde $prefix(u)$ es el prefijo al que representa el nodo $u$.

Podemos denotar el nodo que representa a la cadena vacía con $0$ e ir añadiendo nodos al grafo. Ahora, si consideramos que nuestro alfabeto es $\Sigma$, entonces cada nodo tendrá a lo mucho $|\Sigma|$ aristas, así que podemos almacenar las transiciones de dos maneras:

1. Cuando $|\Sigma|$ es pequeño, podemos almacenar las transiciones en un vector de tamaño fijo $|\Sigma|$ y usar una función que mapee cada caracter del alfabeto a un número entre $0$ y $|\Sigma| - 1$. Esto costará una memoria de $O(n|\Sigma|)$, donde $n$ es la cantidad de nodos del trie (que esta acotado numéricamente por la suma de las longitudes de las cadenas + 1). En este caso, como la transición está almacenada en un vector, acceder a ella toma $O(1)$.

2. Cuando $|\Sigma|$ es grande, podemos almacenar las transiciones en un `std::map`, de esta manera almacenaremos exactamente la cantidad de aristas en total, así que la complejidad espacial será de $O(n)$, pero en este caso, el acceder a una transición tomará $O(\log{\min{\{n, |\Sigma|\}}})$.

Inicialmente el Trie solo tendrá el nodo que representa a la cadena vacía y no tendrá aristas, y luego iremos insertando los strings de nuestro conjunto.

### Inserción

Empezaremos por el nodo que representa a la cadena vacía e iremos iterando sobre los caracteres de la cadena a insertar (sea $s$). 

En cada caracter $s_{i}$ hay dos posibilidades:

1. La transición del nodo $pos$ con etiqueta $s_{i}$ existe: En este caso, nos basta con seguir la transición ya establecida.

2. La transición del nodo $pos$ con etiqueta $s_{i}$ no existe: En este caso, creamos un nodo nuevo $v$, sin transiciones, y establecemos la transición de $pos$ con etiqueta $s_{i}$ a $v$.

Al terminar de procesar todos los caracteres de $s$ llegaremos a un nodo $pos$ que tendrá que ser marcado como **terminal**, ya que existe una cadena en nuestro conjunto que es representada por dicho nodo.

**Nota:** $pos$ es una variable, no es un nodo fijo, por eso lo referenciamos como tal.

```C++
// Consideraremos que tenemos un arreglo trie[n][sigma]
// Y si trie[pos][c] = 0 quiere decir que la transición no existe.
// nodes es la variable que contiene la cantidad de nodos del grafo

int nodes = 1; // Inicialmente tenemos al nodo 0

void insert(string &s){
    int pos = 0;
    for(int i = 0; i < s.size(); i++){
        int nxt = mapeo(s[i]);
        if(trie[pos][nxt] == 0){
            trie[pos][nxt] = nodes++; // Creamos un nodo
        }
        pos = trie[pos][nxt]; // Avanzamos por la transicion
    }
    terminal[pos] = true; // Marcamos el nodo como terminal
}
```

Notemos que la función $mapeo(c)$ nos da un entero entre $0$ y $|\Sigma| - 1$ como describimos anteriormente.

La complejidad de esta acción es $O(|s|)$.

### Búsqueda

Para poder buscar una cadena dentro de nuestro trie, nos basta con iterar sobre los caracteres y usar las transiciones establecidas; es evidente que si no hay alguna de las transiciones, la cadena no pertenecerá al conjunto.

```C++
bool search(string &s){
    int pos = 0;
    for(int i = 0; i < s.size(); i++){
        int nxt = mapeo(s[i]);
        if(trie[pos][nxt] == 0) return false;
        pos = trie[pos][nxt]; // Avanzamos por la transicion
    }
    return true;
}
```

La complejidad de esta acción es $O(|s|)$.

### Eliminación

Para poder *eliminar* una cadena (bajo la premisa de que esta está en nuestro conjunto del Trie), podemos extender la información que tiene cada nodo:

Como cada nodo representa a un prefijo de alguna de las cadenas, podemos considerar que mantenemos el conteo de cuántas veces aparece cada prefijo, de forma que si eliminamos la cadena $s$, tendremos que restar 1 a las frecuencias de los prefijos de $s$ dentro del Trie.

```C++
// Declaramos un arreglo frec[n] que tendrá
// la frecuencia de cada nodo como prefijo

void erase(string &s){
    int pos = 0;
    for(int i = 0; i < s.size(); i++){
        int nxt = mapeo(s[i]);
        frec[pos] -= 1;
        pos = trie[pos][nxt]; // Avanzamos por la transicion
    }
    frec[pos] -= 1;
}
```

La complejidad de esta acción es $O(|s|)$.

Esta extensión de la estructura requiere una modificación en la función de búsqueda: Ahora no solo nos fijaremos si la transición existe, sino que también verificaremos que la frecuencia del nodo de la transición no sea 0.

```C++
bool search(string &s){
    int pos = 0;
    for(int i = 0; i < s.size(); i++){
        int nxt = mapeo(s[i]);
        if(trie[pos][nxt] == 0 or frec[trie[pos][nxt]] == 0) return false;
        pos = trie[pos][nxt]; // Avanzamos por la transicion
    }
    return true;
}
```

## Aplicaciones de Trie

### Ordenar un conjunto de strings

Notemos que si usamos un árbol binario de búsqueda para almacenar las etiquetas de las aristas, podremos recorrerlas en orden, por lo que nos bastará insertar las cadenas y luego mandar un DFS sobre el Trie y podremos obtener el orden lexicográfico entre ellas.

```C++
// Insertamos todas las cadenas previamente
// y consideramos que las aristas del trie
// están almacenadas con map<int, int> o map<char, int>

void DFS(int u){
    if(pattern[u]) printf("%d ", u);
    for(auto e : trie[u]){
        DFS(e.second);
    }
}
```

Esto tomará una complejidad de $O(n\cdot\log{|\Sigma|})$ u $O(n)$ dependiendo de la implementación del Trie. Podemos extender este concepto y hallar el k-ésimo string de un conjunto de strings en $O(maxlen)$, donde $maxlen$ es la máxima longitud de entre todos los strings del conjunto.

### Máximo Bitwise XOR

Podemos usar el trie sobre un conjunto de números, todos en base binaria, y considerar que todos tienen una longitud fija $L$.

Si tenemos un entero $x$ fijo con representación binaria de longitud $l \leq L$ (no hay problema si es menor, solamente rellenamos con *leading zeroes*) y deseamos hallar el máximo valor posible de:

$$ M = \max_{y \in S}{\{x \oplus y\}} $$

Entonces podemos usar un algoritmo Greedy para determinar $M$, apoyándonos con la estructura del Trie.

Iteraremos desde el bit de mayor orden ($L$ - 1) hasta el de menor orden ($0$) e iremos intentando "forzar" que el resultado tenga el $i$-ésimo bit prendido.

Esta idea funciona pues si logramos que el $i$-ésimo bit se prenda, entonces su aporte será de:

$$ 2^{i} $$

Y es bien sabido que:

$$ 2^{i} > \sum\limits_{j = 0}^{i - 1} 2^{j} $$

Lo que implica que si prendemos el $i$-ésimo bit tendremos un resultado mayor que si no lo activáramos pero sí prendiéramos los bits siguientes.

```C++
int maximizeXor(int x){
    int ans = 0;
    int pos = 0;
    for(int i = L - 1; i >= 0; i--){
        int have = (x >> i) & 1; // i-ésimo bit de x
        int nxt = have ^ 1; // Inicialmente queremos have ^ 1 para que el bit del XOR esté prendido
        if(trie[pos][nxt] == 0) nxt ^= 1; // Si no hay transición, cambiamos al bit opuesto
        if(trie[pos][nxt] == 0) return -1; // No hay ningun numero en nuestro conjunto
        ans |= (have ^ nxt) << i; // Actualizamos la respuesta
        pos = trie[pos][nxt];
    }
    return ans;
}
```

Esto tendrá una complejidad de $O(L)$.

## Problemas para implementar en clase

- [ADA University - January 9 - Trie](https://www.e-olymp.com/en/contests/15100)
- [Ada and Indexing](https://www.spoj.com/problems/ADAINDEX/en/)
- [Old Berland Language](https://codeforces.com/contest/37/problem/C)
- [SubXor](https://www.spoj.com/problems/SUBXOR/)
- [Sausage Maximization](https://codeforces.com/problemset/problem/282/E)
- [Query On Strings](https://www.codechef.com/problems/NPLFLF)
- [IOI '08 P1 - Type Printer](https://dmoj.ca/problem/ioi08p1)
- [Boruvka volvió](https://codeforces.com/problemset/problem/888/G)


# Aho-Corasick Automaton

El autómata de Aho-Corasick nos permite preprocesar un conjunto de cadenas $S$ para luego poder procesar un texto $s$ y buscar el máximo prefijo de alguna de las cadenas de $S$ que esté contenido en cada prefijo de $s$ como sufijo, todo con una complejidad lineal.

## Suffix Links

Los **suffix links** en el algoritmo Knuth-Morris-Pratt (KMP) se definen como el máximo sufijo **propio** (es decir, no puede ser la cadena completa) que es prefijo de la cadena $s$ analizada al procesar su función $\pi(i)$. Esto se puede visualizar como tomar el sufijo actual e irle quitando elementos al inicio hasta que se vuelva un prefijo también.

Ejemplo:

$$ s = ``abcabcabd'' $$

Es evidente que $\pi(8) = 5$, dado que $abcab$ es el máximo sufijo de $s[1,8]$ que es prefijo de la misma.

Ahora, los suffix links en un conjunto de strings $S$ se definen de manera similar, ahora ya no es el máximo sufijo **propio** que sea prefijo de la misma cadena, sino que sea prefijo de alguna cadena de $S$.

Podemos considerar que el autómata del KMP es un caso especial del autómata de Aho-Corasick con una sola cadena.

## Construcción

Para construir el Autómata del Aho-Corasick, debemos considerar que almacenaremos el conjunto de strings $S$ en un Trie, esto quiere decir que cada nodo es un prefijo de alguna cadena de $S$ (observación clave relacionada con la definición de los suffix links).

Ahora, la definición de un autómata determinista es que cada estado debe tener una transición definida por cada letra del alfabeto $\Sigma$, lo que implica que nuestro Trie muy probablemente no sea aún un autómata. Definamos la función de transición como $go(u,c)$, que nos dará la transición del nodo/estado $u$ usando la letra $c$ (etiqueta de la arista). Además, supongamos que $suffix(u)$ es el suffix link del nodo/estado $u$, $par(u)$ el padre de $u$ en el trie y $enter(u)$ es la letra $l$ tal que `trie[par[u]][l] = u`.

Claramente, deseamos rescatar lo más posible de nuestra cadena analizada, por lo que el $go(u,c)$ nos debe llevar al estado que represente la máxima cadena que es sufijo de $path(u) + c$ y sea prefijo de alguna cadena de $S$.

Entonces, ¿Cómo calcular las tablas $go$ y $suffix$?. Para ello, debemos considerar la siguiente proposición:

**Proposición 1:**

Si itero sobre todos los suffix link de un nodo de manera recursiva hasta que quede la cadena vacía, obtendré todos los sufijos del nodo $u$ que son prefijo de alguna cadena de $S$.

**Prueba:**

Primero, recordemos que los suffix link nos dirigen a un sufijo propio, por lo que su longitud será como máximo $(L-1)$ si el nodo $u$ representa una cadena de longitud $L$, por lo que iteraremos a lo mucho $L$ veces y nuestra iteración es finita.

Segundo, asumamos que no obtenemos todos los sufijos correctos, es decir, existe un sufijo de longitud $x$ tal que no pertenece a nuestra secuencia $L_{1} > L_{2} > L_{3} > \ldots > L_{k} \geq 0$ y además es un prefijo de alguna cadena de $S$. Sin pérdida de generalidad, supongamos que se cumple $L_{i} > x > L_{i+1}$, pero esto implica que el máximo sufijo propio de $L_{i}$ correcto es $x$ y no $L_{i+1}$ como se propone al inicio. Esto es una contradicción, por lo que concluimos que si visitamos todos los sufijos correctos.

### Función $go$

Ahora podríamos analizar lo siguiente respecto a la función $go(u,c)$:

1. Si `trie[u][c]` está definido, entonces $go(u,c) = trie[u][c]$. Esto es válido pues el nodo $trie[u][c]$ representa al prefijo de máxima longitud contenido como sufijo en el prefijo actual del texto.

2. Si no hay una transición directa y $u = raiz = 0$, entonces la máxima cadena que es sufijo de $path(u) + c$ y sea prefijo de alguna cadena de $S$ es la misma cadena vacía, por lo que $go(u,c) = 0$.

3. Si no se cumple ninguna de las anteriores, debemos obtener la transición que nos daría el nodo $v = suffix(u)$ con la letra $c$, es decir, $go(u,c) = go(suffix(u),c)$. Esto debido a la proposición 1.

$$ path(u) + c \rightarrow path(suffix(u)) + c $$

Notemos que $path(suffix(u))$ es el máximo sufijo propio de $path(u)$ que es prefijo de alguna cadena de $S$, por lo que es nuestra mejor opción posible en ese momento. La recursividad garantiza encontrar (eventualmente) algún sufijo que funcione.

### Función $suffix$

Al paralelo, considerando que nuestra función $go(u,c)$ está bien definida, podemos analizar la función $suffix(u)$:

1. Si `par[u] = raiz = 0` eso quiere decir que el único sufijo propio es la cadena vacía, entonces $suffix(u) = 0$.

2. Si no se da el caso anterior, debemos considerar que $path(u) = path(par[u]) + enter(u)$, por lo que el suffix link de $u$ será la transición del suffix link de $par[u]$ usando la letra $enter(u)$, es decir, $suffix(u) = go(suffix[par[u]],enter[u])$.

Siempre podemos visualizar los suffix link como quitar letras al inicio de la cadena hasta que sea prefijo de alguna cadena del conjunto, por lo que es mejor verificarlo de esa forma:

$$ path(suffix(u)) = path(suffix(par[u])) + enter[u] $$

Notemos que si analizamos letra por letra $enter[u]$ usando la recursividad, encontraremos un sufijo de $path(u)$ (debido a que al ascender en los suffix links de los padres se irá formando con las letras finales de $path(u)$. Esto se puede probar por inducción), por lo que terminaremos agregando la máxima cantidad posible de letras (dado que son transiciones definidas) y se hallará correctamente el suffix link.

## Aplicaciones

- Buscar las ocurrencias de patrones $p_{i}$ en un texto largo $t$. Hay solución $O(n + \text{ocurrencias diferentes})$ definiendo *super links* (suffix link hacia un nodo terminal/de aceptación del autómata). También hay una solución $O(n + m)$ usando DP para propagar frecuencias de los nodos. $n = \sum\limits_{i=1}^{m}|p_{i}|$.

- Hallar todas la mínima cadena que contenga un conjunto de strings. Hay solución $O(n \cdot 2^{m} \cdot |\Sigma|)$ usando DP y bitmask.

- Cantidad de cadenas diferentes de longitud $L$ que contengan al menos una de las cadenas $p_{i}$. Dado que es un autómata, debemos extenderlo a un grafo para resolverlo en $O(n^{3}\log_{2}{L})$.

- Determinar si existe alguna cadena infinita que no contenga ninguna de las cadena $p_{i}$. Verificar si existe un ciclo de aristas que no pasen por ningún nodo que tenga *super link*.

- Hallar la k-ésima cadena lexicográfica de tamaño $L$ (o determinar que no la hay) que contenga al menos una de las cadenas $p_{i}$. Usando DP se obtiene una solución $O(L\cdot n\cdot |\Sigma|)$.

## Implementación Recursiva

Definiremos algunas variables:

```C++
const int SUML = 1000000 + 5; // SUML > suma de longitudes de las cadenas
const int E = 26; // Sigma

int n;
int nodes; // Nodos en el trie/automata
int p[SUML]; // Padre del nodo u en el trie
int suf[SUML]; // Suffix link para cada nodo
int frec[SUML]; // Frecuencia de cada nodo como string en S
int total[SUML]; // Cuantos patrones estan contenidos en cada nodo
int enter[SUML]; // Caracter de la transición de p(u) a u
int go[SUML][E]; // Función go
int super[SUML]; // Suffix Link de mayor longitud que es terminal
int trie[SUML][E]; // Transiciones del Trie
```

Función insert para inicializar el trie, considerando que debemos almacenar información extra:

```C++
void insert(string &s){
	int pos = 0;
	for(int i=0; i < s.size(); i++){
		int nxt = s[i] - 'a';
		if(trie[pos][nxt] == 0){
			p[nodes] = pos;
			enter[nodes] = nxt;
			trie[pos][nxt] = nodes++;
		}
		pos = trie[pos][nxt];
	}
	frec[pos] += 1;
}
```

Funciones $go$ (aquí `getGo` para poder usar `go` como arreglo) y $suffix$ (aqui `getSuffixLink`), declarando $suffix$ previamente para que no haya errores en la sintaxis (extras: función $super$ para obtener el super link y $total$ para saber cuántos patrones diferentes hay en un nodo):

```C++
int getSuffixLink(int);

int getGo(int u, int c){
	if(go[u][c] != -1) return go[u][c];
	if(trie[u][c] > 0) return go[u][c] = trie[u][c];
	if(u == 0) return go[u][c] = 0;
	return go[u][c] = getGo(getSuffixLink(u),c);
}

int getSuffixLink(int u){
	if(suf[u] != -1) return suf[u];
	if(p[u] == 0) return suf[u] = 0;
	return suf[u] = getGo(getSuffixLink(p[u]),enter[u]);
}

int getSuperLink(int u){
	if(super[u] != -1) return super[u];
	if(u == 0) return super[u] = 0;
	if(frec[getSuffixLink(u)]) return super[u] = getSuffixLink(u);
	return super[u] = getSuperLink(getSuffixLink(u));
}

int getTotal(int u){
	if(u == 0) return total[u] = 0;
	int superLink = getSuperLink(u);
	return total[u] = frec[u] + total[getSuperLink(u)];
}
```

Función para construir el autómata:

```C++
void buildAutomaton(int n_pat){
	memset(go, -1, sizeof go);
	memset(suf, -1, sizeof suf);
	memset(super, -1, sizeof super);
	nodes = 1;
    string s;
	for(int i = 0; i < n_pat; i++){
		cin >> s;
		insert(s);
	}
	for(int i = 0; i < nodes; i++){
		getSuffixLink(i);
		for(int j = 0; j < E; j++){
			getGo(i, j);
		}
		getSuperLink(i);
		getTotal(i);
	}
}
```

## Implementación Iterativa

Si consideramos los pasos realizados en la implementación iterativa, podemos notar que toda la inicialización de la información, luego de insertar las cadenas, se puede obtener con un BFS. De esta manera, la construcción del autómata cambia a:

```C++
void buildAutomaton(int npat){
	nodes = 1;
    string s;
	for(int i = 0; i < npat; i++){
        cin >> s;
		insert(s);
	}
	BFS(0);
}
```

Y el BFS para rellenar los arreglos $suf$ y $go$ sería así:

```C++
void BFS(int src){
	queue<int> Q;
	Q.emplace(src);
	while(!Q.empty()){
		int u = Q.front();
		Q.pop();
		for(int i = 0; i < E; i++){
			if(trie[u][i]){
				int v = trie[u][i];
                go[u][i] = trie[u][i];
				suf[v] = u ? trie[suf[u]][i] : 0;
				Q.emplace(v);
			}
			else{
				go[u][i] = u ? trie[suf[u]][i] : 0;
			}
		}
	}
}
```

Ambas implementaciones tienen la misma complejidad si usan el mismo almacenamiento para ambos Trie, pero la versión iterativa permite usar menos memora y hasta nos permite *reciclar* el arreglo $trie$ y usarlo como $go$.

## Problemas para practicar:

- [Contest de Aho Corasick &mdash; kuangbin](https://vjudge.net/contest/332744): pass: cocontest6. Los problemas del SPOJ mándenlos en la misma página.